# 1. Import Required Libraries

In [ ]:
# 1. Import required libraries

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from skimage.feature import local_binary_pattern, hog, canny
from skimage.exposure import equalize_hist
from skimage.filters import gaussian
from skimage import img_as_float
from skimage.measure import label

from torchvision.datasets import EMNIST
from torchvision import transforms

import torch
import random


# 2. Data Acquisition – Load EMNIST Letters

In [ ]:
# 2. Data acquisition: load EMNIST Letters

# Transform: just convert to tensor in [0,1], no augmentation
transform = transforms.Compose([
    transforms.ToTensor()
])

# Download EMNIST letters split
train_dataset = EMNIST(
    root="data",
    split="letters",
    train=True,
    download=True,
    transform=transform
)

test_dataset = EMNIST(
    root="data",
    split="letters",
    train=False,
    download=True,
    transform=transform
)

len(train_dataset), len(test_dataset)


100%|██████████| 562M/562M [00:02<00:00, 272MB/s]


# 2.1 Convert Datasets to Numpy & (Optionally) Sample 10k–20k Images

In [ ]:
# Helper: convert a torchvision dataset to numpy arrays
def dataset_to_numpy(dataset, max_samples=None, seed=42):
    rng = np.random.RandomState(seed)
    n = len(dataset)
    if max_samples is not None and max_samples < n:
        indices = rng.choice(n, size=max_samples, replace=False)
    else:
        indices = np.arange(n)

    images = []
    labels = []
    for idx in indices:
        img_tensor, label = dataset[idx]   # img_tensor: [1, 28, 28]
        img = img_tensor.squeeze(0).numpy()  # -> (28, 28)
        images.append(img)
        labels.append(label)

    return np.array(images), np.array(labels)

# Take 20k samples from train data
X_images, y_raw = dataset_to_numpy(train_dataset, max_samples=20000)

X_images.shape, y_raw.shape


# 2.2 Map Labels to A–Z and Show Distribution

In [ ]:
# EMNIST letters labels are 1-26; we map them to 0-25 internally
y = y_raw - 1  # now in [0, 25]

# Helper to map numeric label to actual letter (A-Z)
def idx_to_char(idx):
    return chr(ord('A') + idx)

# Dataset size and category-wise count
unique, counts = np.unique(y, return_counts=True)
print("Total samples:", len(y))
print("Number of classes:", len(unique))

for cls, cnt in zip(unique, counts):
    print(f"Class {cls} ({idx_to_char(cls)}): {cnt}")

# Plot label distribution
plt.figure(figsize=(10, 4))
plt.bar([idx_to_char(i) for i in unique], counts)
plt.xlabel("Letter Class")
plt.ylabel("Number of Samples")
plt.title("Label Distribution in EMNIST Letters (Sampled)")
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.show()


# 3. Data Preparation – Train/Test Split

In [ ]:
# 3. Train-test split (stratified)

X_train_imgs, X_test_imgs, y_train, y_test = train_test_split(
    X_images,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

X_train_imgs.shape, X_test_imgs.shape


# 4. Preprocessing & Feature Engineering

In [ ]:
# 4. Feature engineering

# Parameters for LBP
LBP_RADIUS = 1
LBP_POINTS = 8 * LBP_RADIUS
LBP_METHOD = "uniform"

def extract_features_single(img_28x28):
    """
    img_28x28: numpy array (28x28), values in [0,1] or [0,255]
    Returns: 1D feature vector (np.array)
    """
    # Ensure float in [0,1]
    img = img_as_float(img_28x28)

    # --- Low-level preprocessing ---

    # 1) Histogram equalization (improves contrast)
    img_eq = equalize_hist(img)

    # 2) Gaussian smoothing to reduce noise
    img_smooth = gaussian(img_eq, sigma=1)

    # --- Mid-level features ---

    # A) Edge map using Canny
    edges = canny(img_smooth, sigma=1)
    edge_density = edges.mean()  # fraction of edge pixels

    # B) Connected components on edge image
    labeled = label(edges)
    num_components = labeled.max()  # 0 if no edges

    # C) LBP texture descriptor
    lbp = local_binary_pattern(img_smooth, LBP_POINTS, LBP_RADIUS, method=LBP_METHOD)
    n_bins = LBP_POINTS + 2  # uniform LBP
    lbp_hist, _ = np.histogram(
        lbp.ravel(),
        bins=n_bins,
        range=(0, n_bins),
        density=True  # normalized histogram
    )

    # D) HOG descriptor (gradient-based shape descriptor)
    hog_feat = hog(
        img_smooth,
        orientations=9,
        pixels_per_cell=(7, 7),
        cells_per_block=(2, 2),
        block_norm="L2-Hys",
        transform_sqrt=True,
        feature_vector=True
    )

    # Concatenate all features into a single vector
    feature_vector = np.hstack([
        lbp_hist,
        hog_feat,
        [edge_density, num_components]
    ])

    return feature_vector


# 4.2 Apply Feature Extraction to All Train & Test Images

In [10]:
# Apply feature extraction to all images
def build_feature_matrix(images):
    features = [extract_features_single(img) for img in images]
    return np.vstack(features)

X_train = build_feature_matrix(X_train_imgs)
X_test = build_feature_matrix(X_test_imgs)

X_train.shape, X_test.shape


/usr/local/lib/python3.12/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


((16000, 336), (4000, 336))

# 5. Model Building – kNN, SVM, RF, Logistic Regression

In [11]:
# 5. Model building

models = {
    "kNN": make_pipeline(
        StandardScaler(),
        KNeighborsClassifier(n_neighbors=5)
    ),
    "SVM (RBF)": make_pipeline(
        StandardScaler(),
        SVC(kernel="rbf", C=10, gamma="scale")
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ),
    "LogisticRegression": make_pipeline(
        StandardScaler(),
        LogisticRegression(
            max_iter=1000,
            multi_class="multinomial",
            n_jobs=-1
        )
    )
}

models


{'kNN': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('kneighborsclassifier', KNeighborsClassifier())]),
 'SVM (RBF)': Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC(C=10))]),
 'RandomForest': RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42),
 'LogisticRegression': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression',
                  LogisticRegression(max_iter=1000, multi_class='multinomial',
                                     n_jobs=-1))])}

# 5.1 Train Models and Evaluate via Cross-Validation (on Train Set)

In [12]:
cv_results = {}

for name, model in models.items():
    print(f"\n=== {name} ===")
    # 3-fold cross-validation on training data
    cv_scores = cross_val_score(model, X_train, y_train, cv=3, n_jobs=-1)
    print("CV scores:", cv_scores)
    print("Mean CV accuracy:", cv_scores.mean())
    cv_results[name] = cv_scores.mean()



=== kNN ===
CV scores: [0.85451819 0.85467842 0.85036565]
Mean CV accuracy: 0.853187416826975

=== SVM (RBF) ===


KeyboardInterrupt: 

# 5.2 Fit Models on Full Training Set & Evaluate on Test Set

In [ ]:
test_accuracies = {}

for name, model in models.items():
    print(f"\nTraining {name} on full training data...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    test_accuracies[name] = acc
    print(f"Test accuracy: {acc:.4f}")


# 5.3 Compare Model Accuracies in a Bar Plot

In [ ]:
# Combine CV and test accuracy into a simple bar plot
model_names = list(models.keys())
cv_vals = [cv_results[m] for m in model_names]
test_vals = [test_accuracies[m] for m in model_names]

x = np.arange(len(model_names))
width = 0.35

plt.figure(figsize=(8, 4))
plt.bar(x - width/2, cv_vals, width, label="CV Accuracy")
plt.bar(x + width/2, test_vals, width, label="Test Accuracy")
plt.xticks(x, model_names, rotation=15)
plt.ylabel("Accuracy")
plt.title("Model Accuracies on EMNIST Letters (Handcrafted Features)")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.show()


# 6. Detailed Metrics for the Best Model

In [ ]:
# Find best model by test accuracy
best_model_name = max(test_accuracies, key=test_accuracies.get)
best_model = models[best_model_name]

print(f"Best model: {best_model_name}, Test accuracy = {test_accuracies[best_model_name]:.4f}")

# Predict on test set
y_pred_best = best_model.predict(X_test)

print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred_best,
    target_names=[idx_to_char(i) for i in range(26)]
))


# 6.1 Confusion Matrix Heatmap

In [ ]:
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(8, 8))
plt.imshow(cm, interpolation="nearest")
plt.title(f"Confusion Matrix - {best_model_name}")
plt.colorbar()
tick_marks = np.arange(26)
plt.xticks(tick_marks, [idx_to_char(i) for i in range(26)], rotation=90)
plt.yticks(tick_marks, [idx_to_char(i) for i in range(26)])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
plt.show()


# 7. Model Inference & Evaluation – Show 5 Random Test Images

In [ ]:
# 7. Visual evaluation on 5 random test images

def show_random_predictions(num_samples=5):
    """
    Pick 'num_samples' random test images and display Actual vs Predicted labels.
    Each call gives different images.
    """
    # Randomly pick indices
    indices = np.random.choice(len(X_test_imgs), size=num_samples, replace=False)

    plt.figure(figsize=(12, 3))

    for i, idx in enumerate(indices):
        img = X_test_imgs[idx]
        true_label = y_test[idx]

        # Extract features & predict
        feat = extract_features_single(img).reshape(1, -1)
        pred_label = best_model.predict(feat)[0]

        # Plot image
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(img, cmap="gray")
        plt.axis("off")
        plt.title(f"True: {idx_to_char(true_label)}\nPred: {idx_to_char(pred_label)}")

    plt.suptitle(f"Random Test Predictions ({best_model_name})")
    plt.tight_layout()
    plt.show()


show_random_predictions()


# 8. Validation of Actual Test – Your Own Handwritten Letter

## 8.1 Function to Process an External Image

In [ ]:
from PIL import Image

def preprocess_external_image(path, invert=True):
    """
    Load an external handwritten letter image, convert it to the same format
    as EMNIST, and extract features.
    """
    # Load image with PIL
    img = Image.open(path).convert("L")  # convert to grayscale

    # Resize to 28x28 (EMNIST size)
    img = img.resize((28, 28))

    # Convert to numpy array
    img_np = np.array(img).astype("float32")

    # Depending on background (white vs black), we may need to invert
    if invert:
        img_np = 255.0 - img_np

    # Normalize to [0,1]
    img_np = img_np / 255.0

    return img_np

def predict_external_image(path, invert=True):
    img_28x28 = preprocess_external_image(path, invert=invert)
    features = extract_features_single(img_28x28).reshape(1, -1)
    pred_label = best_model.predict(features)[0]
    pred_char = idx_to_char(pred_label)

    plt.figure(figsize=(3, 3))
    plt.imshow(img_28x28, cmap="gray")
    plt.axis("off")
    plt.title(f"Predicted: {pred_char}")
    plt.show()

    return pred_char

# Example usage (once you have the image file in your environment):
# my_pred = predict_external_image("my_letter.png", invert=True)
# print("Model prediction for my handwritten sample:", my_pred)


In [ ]:
# Example usage (once you have the image file in your environment):
my_pred = predict_external_image("a.png", invert=True)
print("Model prediction for my handwritten sample:", my_pred)

In [ ]:
my_pred = predict_external_image("b.png", invert=True)
print("Model prediction for my handwritten sample:", my_pred)

In [ ]:
my_pred = predict_external_image("ma.png", invert=True)
print("Model prediction for my handwritten sample:", my_pred)

In [ ]:
my_pred = predict_external_image("k.png", invert=True)
print("Model prediction for my handwritten sample:", my_pred)

In [13]:
my_pred = predict_external_image("wpng.png", invert=True)
print("Model prediction for my handwritten sample:", my_pred)

FileNotFoundError: [Errno 2] No such file or directory: 'wpng.png'